In [1]:
import logging
import os

from bs4 import BeautifulSoup, SoupStrainer, Tag
from urllib.request import urlopen
import urllib
import pandas as pd
import time
from urllib.request import Request, urlopen

## Per Page Recipe Scraping

In [ ]:
url =  \
"https://www.tarladalal.com/sabudana-pakoras-14757r"
# "https://www.tarladalal.com/chakli-instant-chakli-40487r"
# "https://www.tarladalal.com/pressure-cooker-oondhiya-4337r"
# "https://www.tarladalal.com/bajra-dhebra-recipe-gujarati-tea-time-snack-42272r"
# "https://www.tarladalal.com/sabudana-khichdi-in-microwave-2749r"
# "https://www.tarladalal.com/sweet-shakarpara-baked-sweet-shakar-para-41973r"
# "https://www.tarladalal.com/vegetable-stuffed-upma-cutlet-36118r"
# "https://www.tarladalal.com/surti-undhiyu-40596r"
# "https://www.tarladalal.com/cauliflower-nu-bhanolu--gujarati-recipe-560r"
# "https://www.tarladalal.com/instant-lemon-pickle--zero-oil-22184r"
# "https://www.tarladalal.com/sprouts-spring-onion-and-tomato-salad-2957r" 
# "https://www.tarladalal.com/khandvi-gujarati-snack-recipe-557r" 
# "https://www.tarladalal.com/sabudana-khichdi-in-microwave-2749r"
hdr = {'User-Agent':'Mozilla/5.0'}
header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}
req = urllib.request.Request(url=url, headers=header) 
page = urllib.request.urlopen(req).read()
soup = BeautifulSoup(page.decode("utf-8", "html.parser"))

webpage = urlopen(req).read()
soup = BeautifulSoup(webpage.decode("utf-8", "html.parser"))

In [ ]:
with open("/home/shalin/t.html", 'w') as f:
    f.write(webpage)

In [ ]:
ing_sections = soup.find_all('span', id=lambda x: x and x.startswith('ingsection'))

recipes_ingredients = ""
# Iterate over each 'ingsection' and print its content
for section in ing_sections:
    if not recipes_ingredients:
        recipes_ingredients += section.text.strip() + ":"
    else:
        recipes_ingredients += ";" + section.text.strip() + ":"
    next_sibling = section.find_next_sibling()  # Get the next sibling
    siblings = []
    while next_sibling:
        if next_sibling.name == 'span' and next_sibling.has_attr('class') and 'recipe_subheader' in next_sibling['class']:
            break  # Break if next sibling is another 'ingsection'
        if next_sibling.text.strip():
            siblings.append(next_sibling.text.strip())
        next_sibling = next_sibling.find_next_sibling()  # Move to the next sibling
    recipes_ingredients += ", ".join(siblings)
recipes_ingredients

In [ ]:
soup.find('span', id='ctl00_cntrightpanel_lblrecipeNameH2').text

In [ ]:
# TODO: For https://www.tarladalal.com/vegetable-stuffed-upma-cutlet-36118r, there's section in the recipe steps. Needs sections in the recipes too
recipe_small_steps_div = soup.find('div', id='recipe_small_steps')
# recipe_small_steps_div.find_all('span')
for span in recipe_small_steps_div.find_all('li'):
    print(span.get_text())

In [ ]:
recipe_small_steps_subheaders = soup.find_all(class_='recipe_subheader')
recipe_small_steps_subheaders

In [ ]:
print(get_ingrediants(soup, url))

In [ ]:
rdiv = soup.find('div', id="rcpinglist")
for sp in rdiv.select('span'):
    if sp.has_attr('itemprop'):
        print(sp.text.strip())
    elif sp.has_attr('class') and 'recipe_subheader' in sp['class']:
        print(sp.text.strip() + ":")

In [ ]:
rec = soup.find('div', id="recipe_small_steps").get_text("\n").strip()
print(rec)
print()
print("\n".join(list(dict.fromkeys(rec.split("\n")))))

In [ ]:
recipe_small_steps_subheaders = soup.find_all(class_='recipe_subheader')
steps = []
for subheader in recipe_small_steps_subheaders:
    if subheader.has_attr('id') and subheader['id'].startswith('procsection'):
        # print(subheader)
        print(subheader.text.strip())  # Print subheader text
        # Find the parent ol element
        parent_ol = subheader.find_next('ol')
        # Find all li elements under the parent ol
        recipe_steps = parent_ol.find_all('li')
        # Extract and print the text for each step
        for step in recipe_steps:
            print(step.span.text.strip())
            steps.append(step.span.text.strip())
if not steps:
    recipe_small_steps_div = soup.find('div', id='recipe_small_steps')
    # recipe_small_steps_div.find_all('span')
    for span in recipe_small_steps_div.find_all('li'):
        print(span.get_text())
        steps.append(span.get_text())
steps

In [ ]:
parent_span = soup.find('span', id='ctl00_cntrightpanel_lblServes')
# text_in_parent_span = ''.join([str(x) for x in parent_span.contents if isinstance(x, str)])
print(text_in_parent_span)

In [ ]:
time_tag = soup.find('time', itemprop='totalTime')
print(time_tag)
span_tag = time_tag.find('span')
if span_tag:
    span_tag.decompose()
print(time_tag.text)

In [ ]:
soaking_time_element = soup.find(text=lambda text: 'Soaking time' in text)
print(soaking_time_element.split(':')[1].split('\xa0')[1])
# Extract the soaking time value
if soaking_time_element:
    soaking_time = soaking_time_element.split(':')[1].split('\xa0')[1].strip()
    print("Soaking time:", soaking_time)
else:
    print("Soaking time not found")

In [ ]:
soup.find('span', id='ctl00_cntrightpanel_lblrecipeNameH2').text

In [ ]:
print(soup.find('div', id='recipe_tags').get_text("\n").strip())

In [ ]:
get_time_and_serving(soup, url)

In [ ]:
tags_div = soup.find('div', id='recipe_tags')
p_tag = tags_div.find_next('p')
for t in p_tag.find_all('time'):
    t.decompose()
for serv_time in p_tag.find_all('span', attrs=dict(id="ctl00_cntrightpanel_lblServes")):
    serv_time.decompose()
print(p_tag.get_text().
      replace("Preparation Time:", "").
      replace("Cooking Time:", "").
      replace("Total Time:", "").
      replace("&nbsp", "").
      split("Show me for")[0].strip())

## Page Links Scraping

In [ ]:
links_url = "https://www.tarladalal.com/recipes-for-veg-recipes-gujarati-24?pageindex=1" #"https://www.tarladalal.com/sprouts-spring-onion-and-tomato-salad-2957r" # "https://www.tarladalal.com/sabudana-khichdi-in-microwave-2749r" #
links_header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
      'AppleWebKit/537.11 (KHTML, like Gecko) '
      'Chrome/23.0.1271.64 Safari/537.11',
      'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
      'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
      'Accept-Encoding': 'none',
      'Accept-Language': 'en-US,en;q=0.8',
      'Connection': 'keep-alive'}
links_req = urllib.request.Request(url=links_url, headers=links_header) 
links_page = urllib.request.urlopen(links_req).read()

In [ ]:
links_soup = BeautifulSoup(links_page.decode("utf-8", "html.parser"))

In [ ]:
links_soup

In [ ]:
for div in links_soup.findAll('div', class_="rcc_rcpcore"):
    print(div)
    print("**********************************")

In [ ]:
for sp in links_soup.find_all('span', attrs={"class":"rcc_recipename"}):
    print(sp.text)
    print(sp.find('a', href=True)['href'])

# Final Scrapper

In [2]:
def get_webpage(links_url):
    try:
        links_header= {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
          'AppleWebKit/537.36 (KHTML, like Gecko) '
          'Chrome/123.0.0.0 Safari/537.36',
          'Accept': '*/*', #'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
          'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
          'Accept-Encoding': 'none',
          'Accept-Language': 'en-US,en;q=0.9',
          'Connection': 'keep-alive',
        }
        links_req = urllib.request.Request(url=links_url, headers=links_header) 
        try:
            links_page = urllib.request.urlopen(links_req, timeout=90).read()
            # print(links_page)
        except urllib.error.HTTPError:
            logging.exception(f"Error when getting webpage from {links_url}")
            time.sleep(120)
            logging.info("Retrying......")
            try:
                links_page = urllib.request.urlopen(links_req, timeout=90).read()
            except:
                logging.warning("Retry Failed!!!")
                return 
        links_soup = BeautifulSoup(links_page.decode("utf-8", "html.parser"))
        return links_soup
    except:
        logging.exception(f"Error when getting webpage code for {links_url}")

In [3]:
def get_ingrediants(soup, page_link):
    recipes_ingredients = ""
    try:
        ing_list = []
        rdiv = soup.find('div', id="rcpinglist")
        for sp in rdiv.select('span'):
            if sp.has_attr('itemprop'):
                ing_list.append(sp.text.strip())
            elif sp.has_attr('class') and 'recipe_subheader' in sp['class']:
                ing_list.append(sp.text.strip() + ":")
        recipes_ingredients = "\n ".join(ing_list)
    except:
        logging.exception(f"Exception when finding ingrediants of {page_link}")
    
    if not recipes_ingredients:
        logging.warning(f"Cannot find recipe ingredients for {page_link}")
    return recipes_ingredients        

In [5]:
def get_recipe(soup, page_link):
    recipe = ""
    try:
        rec = soup.find('div', id="recipe_small_steps").get_text("\n").strip()
        recipe = "\n".join(list(dict.fromkeys(rec.split("\n"))))
    except:
        logging.exception(f"Exception when finding recipe for {page_link}")

    if not recipe:
        logging.warning(f"Cannot find recipe method for {page_link}")
    return recipe

In [4]:
def get_time_and_serving(soup, page_link):
    times = ""
    servings = ""
    other_info = ""
    try:
        prep_time = ""
        cook_time = ""
        total_tie = ""
        
        servings_tag = soup.find('span', attrs=dict(itemprop="recipeYield"))
        if servings_tag:
            servings = str(servings_tag.text)
        
        prep_tag = soup.find('time', attrs=dict(itemprop="prepTime"))
        if prep_tag:
            prep_time = prep_tag.text
            times += "Preparation Time: " + prep_time+ ", "

        cook_tag = soup.find('time', attrs=dict(itemprop="cookTime"))
        if cook_tag:
            cook_time = cook_tag.text
            times += "Cooking Time: " + cook_time + ", "
        
        total_time_tag = soup.find('time', itemprop='totalTime')
        if total_time_tag:
            span_tag = total_time_tag.find('span')
            if span_tag:
                span_tag.decompose()
            total_time = total_time_tag.text
            times += "Total Time: " + total_time + ", "
  
        # Other time or coocking related info
        tags_div = soup.find('div', id='recipe_tags')
        if not tags_div:
            tags_div = soup.find('div', id='ctl00_cntrightpanel_lblrecipeNameH2')
            
        p_tag = tags_div.find_next('p')
        for t in p_tag.find_all('time'):
            t.decompose()
        if servings:
            for serv_time in p_tag.find_all('span', attrs=dict(id="ctl00_cntrightpanel_lblServes")):
                serv_time.decompose()
        
        other_info = p_tag.get_text().replace("&nbsp", " ").split("Show me for")[0].strip()        
        if prep_time:
              other_info = other_info.replace("Preparation Time:", "")
        if cook_time:
              other_info = other_info.replace("Cooking Time:", "")
        if total_time:
              other_info = other_info.replace("Total Time:", "")
        
        times += other_info
    
    except:
        logging.exception(f"Error while finding time and serving for {page_link}") 

    if not times or not servings:
        logging.warning(f"Cannot find times, or servings for {page_link}")
    return times, servings

#### Get the pages and content

In [6]:
cuisine = "mh"
base_data_dir = f"../data/{cuisine}"

In [7]:
recipe_pages_info = {
    "gj": {"list_page_url": "https://www.tarladalal.com/recipes-for-veg-recipes-gujarati-24?pageindex={0}",
          "last_list_page_num": 38
          },
    "mh": {"list_page_url": "https://www.tarladalal.com/recipes-for-veg-recipes-maharashtrian-52?pageindex={0}",
           "last_list_page_num": 21
          }
}

In [23]:
# recipe_list_page_url = "https://www.tarladalal.com/recipes-for-veg-recipes-maharashtrian-52?pageindex={0}" #"https://www.tarladalal.com/recipes-for-veg-recipes-gujarati-24?pageindex={0}"
recipe_url_prefix = "https://www.tarladalal.com/"
for recipe_list_page_idx in range(1, recipe_pages_info[cuisine]["last_list_page_num"]+1):
    recipe_list = []
    print(f"Getting recipies from page {recipe_list_page_idx}")
    list_url = recipe_pages_info[cuisine]["list_page_url"].format(recipe_list_page_idx)
    recipe_list_page = get_webpage(list_url)
    if not recipe_list_page:
        continue
    for recipe_span in recipe_list_page.find_all('span', attrs={"class":"rcc_recipename"}):
        df_row = {}
        df_row["recipe_list_url"] = list_url
        df_row["recipe_name"] = recipe_span.text
        recipe_url = recipe_url_prefix + recipe_span.find('a', href=True)['href']
        df_row["recipe_url"] = recipe_url    
        # print(recipe_url)
        recipe_page = get_webpage(recipe_url)
        if not recipdef e_page:
            logging.warn(f"Couldn't find recipe page for {recipe_url}")
            continue
        df_row["recipe_long_name"] = recipe_page.find('span', id='ctl00_cntrightpanel_lblrecipeNameH2').text
        df_row["ingredients"] = get_ingrediants(recipe_page, recipe_url)
        df_row["recipe"] = get_recipe(recipe_page, recipe_url)
        df_row["tags"] = recipe_page.find('div', id='recipe_tags').get_text("\n").strip()
        if not df_row["tags"]:
            logging.warn(f"Couldn't find tags for {recipe_url}")
        times, servings = get_time_and_serving(recipe_page, recipe_url)
        df_row["times"] = times
        df_row["servings"] = servings
        recipe_list.append(df_row)
    recipe_df = pd.DataFrame(recipe_list)
    recipe_df.to_csv(f"{base_data_dir}/recipe_page_{recipe_list_page_idx}.csv", index=False)

SyntaxError: invalid syntax (2852753426.py, line 18)

#### TODO:
Page 1: Need other times in addition of cooking etc time.

All Pages, 
- In recipe, if there's only . then remove it and remove line.
If starts with "To make" then merge next line with it
If starts with , then merge with previous line

In [19]:
def get_recipes_df(page_id):
    pg_location = f"{base_data_dir}/recipe_page_{page_id}.csv"
    if not os.path.exists(pg_location):
        print(f"Page Id Missing: {page_id}")
        return pd.DataFrame()
    return pd.read_csv(pg_location)

In [20]:
for page_id in range(1, recipe_pages_info[cuisine]["last_list_page_num"]+1):
    df = get_recipes_df(page_id)
    if df.empty:
        continue
    print(f"Page Idx: {page_id}")
    for idx, row in df.iterrows():
        if len(df.loc[idx, "recipe_ing"].split("\n")) <= 3:
            print(f'Less ing for {row["recipe_url"]}')
    for idx, row in df.iterrows():
        if len(df.loc[idx, "recipe"].split("\n")) <= 3:
            print(f'Less recipe for {row["recipe_url"]}')

Page Idx: 1
Less ing for https://www.tarladalal.com/how-to-make-rajgira-flour-amaranth-flour-at-home-42738r
Page Idx: 2
Page Idx: 3
Less ing for https://www.tarladalal.com/how-to-make-hung-curd-42765r
Less ing for https://www.tarladalal.com/phulka-recipe-indian-chapati-recipe-4392r
Less ing for https://www.tarladalal.com/jowar-roti-41569r
Page Idx: 4
Page Idx: 5
Page Idx: 6
Less recipe for https://www.tarladalal.com/green-chutney--mumbai-roadside-recipes-33414r
Less recipe for https://www.tarladalal.com/sukhi-lehsun-chutney--mumbai-roadside-recipes--33381r
Page Idx: 7
Less ing for https://www.tarladalal.com/sunny-side-up-eggs-breakfast-recipe-42311r
Page Idx: 8
Less ing for https://www.tarladalal.com/pomegranate-tea-turkish-tea-41740r
Page Idx: 9
Page Idx: 10
Page Idx: 11
Less recipe for https://www.tarladalal.com/geela-lehsun-chutney-33382r
Page Idx: 12
Less ing for https://www.tarladalal.com/how-to-roast-onion-roasted-onions-22531r
Page Idx: 13
Less ing for https://www.tarladalal.com

In [21]:
# Remove tags from tags column
for page_id in range(1, recipe_pages_info[cuisine]["last_list_page_num"]+1):
    df = get_recipes_df(page_id)
    if df.empty:
        continue
    print(f"Page Idx: {page_id}")
    # for idx, row in df.iterrows():
    df['tags'] = df['tags'].str.replace("Tags", "").str.strip()
    df.to_csv(f"{base_data_dir}/recipe_page_{page_id}.csv", index=False)

Page Idx: 1
Page Idx: 2
Page Idx: 3
Page Idx: 4
Page Idx: 5
Page Idx: 6
Page Idx: 7
Page Idx: 8
Page Idx: 9
Page Idx: 10
Page Idx: 11
Page Idx: 12
Page Idx: 13
Page Idx: 14
Page Idx: 15
Page Idx: 16
Page Idx: 17
Page Idx: 18
Page Idx: 19
Page Idx: 20
Page Idx: 21


In [17]:
# Add missing item name in the last line
# Do with 
# Cool the
# Divide the
# Use the
# Store the
# Drain the
# Refrigerate the
# Garnish the
# Break the
# Allow the
# Serve the anywhere then add recipi and merge next line
# If second last word is "Serve" then insert the recipi name and merge last line
half_sent_samples = ["Do with", "Cool the", "Divide the", "Use the", "Store the", "Drain the", "Refrigerate the", 
             "Garnish the", "Break the", "Allow the", "Serve the", "Cut the", "Brush each",
             "Cool and store the", "Serve", "Serve or store the"]
half_sent = half_sent_samples.copy()

for hs in half_sent_samples:
    half_sent.append(hs + " ")

for page_id in range(1, recipe_pages_info[cuisine]["last_list_page_num"]+1):
    df = get_recipes_df(page_id)
    if df.empty:
        continue
    print(f"Page Idx: {page_id}")
    df = df.drop([x for x in  ['Unnamed: 0.1','Unnamed: 0'] if x in df.columns], axis=1)
    for idx, row in df.iterrows():
        recipe = df.loc[idx, "recipe"]
        lines = recipe.split("\n")
        for sentence in half_sent:
            if sentence in lines:
                rn = row["recipe_name"]
                if '(' in rn:
                    rn = rn.split('(')[0].strip()
                elif '|' in rn:
                    rn = rn.split('|')[0].strip()
                elif ',' in rn:
                    rn = rn.split(',')[0].strip()
    
                rn = rn.lower()
                for word in ["raw", "how", "recipe", cuisine, "to", "make"]:
                    if word in rn:
                        rn = rn.replace(word, "")
                ln_idx = lines.index(sentence)
                lines[ln_idx] = f"{lines[ln_idx].strip()} {rn.lower().strip()} {lines[ln_idx+1].strip()}"
                del lines[ln_idx+1]
                new_recipe = "\n".join(lines)
                df.loc[idx, "recipe"] = new_recipe
    df.to_csv(f"{base_data_dir}/recipe_page_{page_id}.csv", index=False)


Page Id Missing: 1
Page Idx: 2
Page Id Missing: 3
Page Id Missing: 4
Page Id Missing: 5
Page Id Missing: 6
Page Id Missing: 7
Page Id Missing: 8
Page Id Missing: 9
Page Id Missing: 10
Page Id Missing: 11
Page Id Missing: 12
Page Id Missing: 13
Page Id Missing: 14
Page Id Missing: 15
Page Id Missing: 16
Page Id Missing: 17
Page Id Missing: 18
Page Id Missing: 19
Page Id Missing: 20
Page Id Missing: 21


In [22]:
for page_id in range(1, recipe_pages_info[cuisine]["last_list_page_num"]+1):
    df = get_recipes_df(page_id)
    if df.empty:
        continue
    print(f"Page Idx: {page_id}")
    df.rename(columns={"recipe_ing":"ingredients"}, inplace=True)
    df.to_csv(f"{base_data_dir}/recipe_page_{page_id}.csv", index=False)

Page Idx: 1
Page Idx: 2
Page Idx: 3
Page Idx: 4
Page Idx: 5
Page Idx: 6
Page Idx: 7
Page Idx: 8
Page Idx: 9
Page Idx: 10
Page Idx: 11
Page Idx: 12
Page Idx: 13
Page Idx: 14
Page Idx: 15
Page Idx: 16
Page Idx: 17
Page Idx: 18
Page Idx: 19
Page Idx: 20
Page Idx: 21


In [ ]:
for page_id in range(1, recipe_pages_info[cuisine]["last_list_page_num"]+1):
    df = get_recipes_df(page_id)
    if df.empty:
        continue
    print(f"Page Idx: {page_id}")
    df = df.drop([x for x in  ['Unnamed: 0.1','Unnamed: 0'] if x in df.columns], axis=1)
    # for idx, row in df.iterrows():
    # df.rename({"recipe_ing":"ingredients"}, axis=1, inplace=True)
    df.to_csv(f"{base_data_dir}/recipe_page_{page_id}.csv", index=False)